In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import acquire
import prepare

### 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [3]:
attendance = pd.read_sql('''SELECT * FROM attendance;''', acquire.get_db_url('tidy_data'))

In [6]:
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


### 2. Coffee Levels

a. Read the coffee_levels table.

In [7]:
coffee_levels = pd.read_sql('''SELECT * FROM coffee_levels;''', acquire.get_db_url('tidy_data'))

In [8]:
coffee_levels

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


b. Transform the data so that each carafe is in it's own column.

In [9]:
coffee_levels.pivot_table(values = 'coffee_amount', index='hour', columns='coffee_carafe')

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


c. Is this the best shape for the data?

### 3. Cake Recipes

a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [10]:
cake_recipes = pd.read_sql('''SELECT * FROM cake_recipes;''', acquire.get_db_url('tidy_data'))

In [11]:
cake_recipes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


b. Tidy the data as necessary.

c. Which recipe, on average, is the best? recipe b

d. Which oven temperature, on average, produces the best results? 275

e. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees